In [4]:
import utils
import os
import openai
openai.api_key = utils.get_openai_api_key()

In [16]:
doc_path="./documents"
files_list=os.listdir(doc_path)
files=[os.path.join(doc_path,file_name) for file_name in files_list]

In [17]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=files
).load_data()

In [18]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

141 

<class 'llama_index.core.schema.Document'>
Doc ID: 68ebcfd7-1573-4929-8549-e58ad1b68b43
Text: Userid: CPM Schema:  i1040xLeadpct: 100% Pt. size: 10  Draft  Ok
to Print AH XSL/XML Fileid: … ions/i1040/2023/a/xml/cycle08/source
(Init. & Date) _______ Page 1 of 114  13:55 - 27-Dec-2023 The type and
rule above prints on all proofs including departmental reproduction
proofs. MUST be removed before printing. Dec 27, 2023 Cat. No. 24811V
Futur ...


In [19]:
## Basic RAG pipeline
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [22]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents([document],service_context=service_context)

/var/folders/s4/2ckbp3v55651lb5c8j__q7dw0000gn/T/ipykernel_35938/3391627733.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


In [23]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [24]:
from utils import get_sentence_window_query_engine
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [27]:
window_response = sentence_window_engine.query(
   "What is the workflow to compute student loan interest deduction? Write points wise")
print(str(window_response))

1. Enter the total interest paid on qualified student loans in 2023, not exceeding $2,500.
2. Multiply the total interest paid by the adjusted gross income.
3. Subtract the result from step 2 from the total interest paid in step 1.
4. Enter the final deduction amount on Schedule 1, line 21.
